In [1]:
import datetime as dt
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm
length_packet = 362

def rtt_plot(df):
    l1 = []
    l2 = []
    for i in tqdm(range(len(df))):
        if not np.isnan(df.loc[i, "tcp.analysis.ack_rtt"]):
            l1.append(df.loc[i, "frame.time"])
            l2.append(df.loc[i, "tcp.analysis.ack_rtt"])
    print("mean:", np.mean(l2))
    plt.plot(l1, l2)
    plt.show()



def get_seq_time():
    rxdf = pd.read_csv(rx_pcap, sep='@')
    rxdf.loc[:, "frame.time"] = pd.to_datetime(rxdf.loc[:, r"frame.time"]).dt.tz_localize(None)
    # map seq to time
    
    rseq_time_list  = []
    tseq_time_list  = []

    for i in tqdm(range(len(rxdf))):
        if rxdf.loc[i, "tcp.len"] >= length_packet and rxdf.loc[i, "ip.src"] == "140.112.20.183":
            bias = rxdf.loc[i, "tcp.payload"].find("0000000062")
            dup_num = (rxdf.loc[i, "tcp.len"] -bias) // length_packet
            packet_time = rxdf.loc[i, "frame.time"]
            for j in range(dup_num):
                seq = int(rxdf.loc[i, "tcp.payload"][bias+32+j*length_packet*2:bias+48+j*length_packet*2], 16)
#                 print(seq)
                rseq_time_list.append((seq, packet_time))

def get_client_ul_seq(txdf):
    length_packet = 362
    tseq_time_dict = {}
    ip_set = set(txdf.loc[:, "ip.src"])
    ip_set.remove("140.112.20.183")
    for i in tqdm(range(len(txdf))):
        if txdf.loc[i, "tcp.len"] >= length_packet and txdf.loc[i, "ip.dst"] == "140.112.20.183":
            bias = txdf.loc[i, "tcp.payload"].find("0000018")
            if bias == -1:
                continue
            dup_num = (txdf.loc[i, "tcp.len"] -bias) // length_packet
            packet_time = txdf.loc[i, "frame.time"]
            for j in range(dup_num):
                seq = int(txdf.loc[i, "tcp.payload"][bias+16+j*length_packet*2:bias+24+j*length_packet*2], 16)
                born_time = int(txdf.loc[i, "tcp.payload"][bias+j*length_packet*2:bias+16+j*length_packet*2], 16)/1000
                born_time = dt.datetime.fromtimestamp(born_time)
                delay = (packet_time - born_time).total_seconds()
                if delay > 0:
                    key = (txdf.loc[i, "ip.src"] , txdf.loc[i, "tcp.srcport"],)
                    if key not in tseq_time_dict:
                        tseq_time_dict[key] = []
                    tseq_time_dict[key].append((seq, packet_time, delay, txdf.loc[i, "frame.number"]))
    for key in tseq_time_dict:
        tseq_time_dict[key].sort(key = lambda x:x[0])
        
    return tseq_time_dict

def get_server_ul_seq(rxdf):
    length_packet = 362
    rseq_time_dict = {}
    for i in tqdm(range(len(rxdf))):
        if rxdf.loc[i, "tcp.len"] >= length_packet:
            bias = rxdf.loc[i, "tcp.payload"].find("0000018")
            if bias == -1:
                continue
            dup_num = (rxdf.loc[i, "tcp.len"] -bias) // length_packet
            packet_time = rxdf.loc[i, "frame.time"]
            for j in range(dup_num):
                seq = int(rxdf.loc[i, "tcp.payload"][bias+16+j*length_packet*2:bias+24+j*length_packet*2], 16)
                born_time = int(rxdf.loc[i, "tcp.payload"][bias+j*length_packet*2:bias+16+j*length_packet*2], 16)/1000
                born_time = dt.datetime.fromtimestamp(born_time)
                delay = (packet_time - born_time).total_seconds()
                if delay > -10:
                    key = (rxdf.loc[i, "ip.src"] , rxdf.loc[i, "tcp.srcport"],)
                    if key not in rseq_time_dict:
                        rseq_time_dict[key] = []
                    rseq_time_dict[key].append((seq, packet_time, delay, rxdf.loc[i, "frame.number"]))
    for key in rseq_time_dict:
        rseq_time_dict[key].sort(key = lambda x:x[0])
        
    return rseq_time_dict

def get_ret(pcapdf):
    # get retransmision time list
    time_list = []
    for i in range(len(pcapdf)):
        if "Retran" in pcapdf.loc[i, "_ws.col.Info"]:
            time_list.append(pcapdf.loc[i, "frame.time"])
    time_list = pd.to_datetime(time_list)
    return time_list

def get_ul_rtt(pcapdf):
    l1 = []
    for i in tqdm(range(len(pcapdf))):
        if not np.isnan(pcapdf.loc[i, "tcp.analysis.ack_rtt"]) and pcapdf.loc[i, "ip.dst"] == "140.112.20.183":
            l1.append((pcapdf.loc[i, "frame.time"], pcapdf.loc[i, "tcp.analysis.ack_rtt"]))
    return np.array(l1)

def get_dl_rtt(pcapdf):
    l1 = []
    for i in tqdm(range(len(pcapdf))):
        if not np.isnan(pcapdf.loc[i, "tcp.analysis.ack_rtt"]):
            l1.append((pcapdf.loc[i, "frame.time"], pcapdf.loc[i, "tcp.analysis.ack_rtt"]))
    return np.array(l1)

def latency_impulse(latency_list, t):
    return sum(latency_list[:, 0] > t) / len(latency_list)

def read_pcapcsv(filename):
    df = pd.read_csv(filename, sep='@')
    df.loc[:, "frame.time"] = pd.to_datetime(df.loc[:, r"frame.time"]).dt.tz_localize(None)
    return df

def UL_latency(rseq_time_list, tseq_time_list):
    latency_list = []
    i = 0
    j = 0
    
    while i < len(rseq_time_list) and j < len(tseq_time_list):
        
        if rseq_time_list[i][0] * 100 < tseq_time_list[j][0] or rseq_time_list[i][0] > tseq_time_list[j][0] * 100:
            break

        if rseq_time_list[i][0] < tseq_time_list[j][0]:
            i += 1
        elif rseq_time_list[i][0] > tseq_time_list[j][0]:
            j += 1
        else:
#             print(rseq_time_list[i], tseq_time_list[j])
            latency_list.append(((rseq_time_list[i][1] - tseq_time_list[j][1]).total_seconds(), tseq_time_list[i][1], rseq_time_list[j][0]))
            i += 1
            j += 1
    latency_list = np.array(latency_list)
    latency_list[:, 0] -= latency_list[:, 0].min()
#     latency_list[:, 0] += 0.01
    plt.title("latency")
    plt.ylabel("seconds")
    plt.plot(latency_list[:, 1], latency_list[:, 0])
    plt.show()
    print(len(latency_list))
    print(np.mean(latency_list[:, 0]))
    return latency_list

def mp_latency(l1, l2):
    # (latency, time, seq)
    
    i = 0
    j = 0
    latency_list = []

    
    while i < len(l1) and j < len(l2):
        
        if l1[i][2] * 100 < l2[j][2] or l1[i][2] > l2[j][2] * 100:
            break

        if l1[i][2] < l2[j][2]:
            i += 1
        elif l1[i][2] > l2[j][2]:
            j += 1
        else:
            print(l1[i], l2[j])
            # latency, time, seq
#             print(l1[i][0], l2[j][0], np.min())
            latency_list.append((np.min((l1[i][0], l2[j][0])), l2[j][1], l2[j][2]))
            i += 1
            j += 1
    latency_list = np.array(latency_list)
#     latency_list[:, 0] -= latency_list[:, 0].min()
#     latency_list[:, 0] += 0.01
    plt.title("latency")
    plt.ylabel("seconds")
    plt.plot(latency_list[:, 0], latency_list[:, 2])
    plt.show()
    print(len(latency_list))
    print(np.mean(latency_list[:, 0]))
    return latency_list



In [12]:
filename = "/home/wmnlab/D/pcap_data/UL_3270_2022-5-10-22-22-7_pcap.csv"
df = read_pcapcsv(filename)

In [13]:
l = get_server_ul_seq(df)

100%|██████████| 5425/5425 [00:00<00:00, 13082.41it/s]


In [14]:
for x in l:
    print(x)

('192.168.1.1', 47556)
('192.168.1.1', 42791)


In [15]:
l[('192.168.1.1', 47556)]

[(0, Timestamp('2022-05-10 22:22:13.630220'), 0.07622, 5),
 (1, Timestamp('2022-05-10 22:22:13.633524'), 0.077524, 7),
 (2, Timestamp('2022-05-10 22:22:13.633524'), 0.076524, 7),
 (3, Timestamp('2022-05-10 22:22:13.633524'), 0.075524, 7),
 (5, Timestamp('2022-05-10 22:22:13.641702'), 0.080702, 11),
 (6, Timestamp('2022-05-10 22:22:13.641702'), 0.079702, 11),
 (7, Timestamp('2022-05-10 22:22:13.641702'), 0.078702, 11),
 (9, Timestamp('2022-05-10 22:22:13.645236'), 0.080236, 13),
 (10, Timestamp('2022-05-10 22:22:13.645236'), 0.078236, 13),
 (11, Timestamp('2022-05-10 22:22:13.645236'), 0.077236, 13),
 (14, Timestamp('2022-05-10 22:22:13.649430'), 0.07743, 19),
 (15, Timestamp('2022-05-10 22:22:13.649430'), 0.07643, 19),
 (16, Timestamp('2022-05-10 22:22:13.649430'), 0.07543, 19),
 (18, Timestamp('2022-05-10 22:22:13.653931'), 0.077931, 23),
 (19, Timestamp('2022-05-10 22:22:13.653931'), 0.075931, 23),
 (20, Timestamp('2022-05-10 22:22:13.653931'), 0.074931, 23),
 (23, Timestamp('2022-05

In [16]:
l[('192.168.1.1', 42791)]

[(226, Timestamp('2022-05-10 22:22:13.921859'), 0.089859, 96),
 (229, Timestamp('2022-05-10 22:22:13.921919'), 0.085919, 98),
 (230, Timestamp('2022-05-10 22:22:13.921919'), 0.084919, 98),
 (231, Timestamp('2022-05-10 22:22:13.921919'), 0.083919, 98),
 (233, Timestamp('2022-05-10 22:22:13.922056'), 0.081056, 100),
 (234, Timestamp('2022-05-10 22:22:13.922056'), 0.080056, 100),
 (235, Timestamp('2022-05-10 22:22:13.922056'), 0.079056, 100),
 (237, Timestamp('2022-05-10 22:22:13.924046'), 0.078046, 102),
 (238, Timestamp('2022-05-10 22:22:13.924046'), 0.077046, 102),
 (239, Timestamp('2022-05-10 22:22:13.924046'), 0.076046, 102),
 (241, Timestamp('2022-05-10 22:22:13.928478'), 0.078478, 104),
 (242, Timestamp('2022-05-10 22:22:13.928478'), 0.076478, 104),
 (243, Timestamp('2022-05-10 22:22:13.928478'), 0.075478, 104),
 (245, Timestamp('2022-05-10 22:22:13.932318'), 0.077318, 106),
 (246, Timestamp('2022-05-10 22:22:13.932318'), 0.076318, 106),
 (247, Timestamp('2022-05-10 22:22:13.932318